# Часть 1 - строки
## 1. Передача данных (0.5)

Для передачи разного рода информации внутри текстовых данных (в частности, с помощью электронной почты), а именно: текст на языках, для которых используются кодировки, отличные от ASCII, и нетекстовые данные, такие, как картинки, музыка, фильмы и программы существуют разные алгоритмы кодирования информации. Вам необходимо реализовать кодирование текстовых последовательностей следующим образом.
#### Кодирование (0.2)
Возьмем текст русский текст «АБВГД». В двоичной форме в кодировке Windows-1251 мы получим 5 байтов:

11000000, 11000001, 11000010, 11000011, 11000100, (00000000) — лишний нулевой байт нужен, чтобы общее число бит делилось на 6.

Разделим эти биты на группы по 6:

110000, 001100, 000111, 000010, 110000, 111100, 010000, 000000.

Берем массив символов

«ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz0123456789+/»

и получившиеся числа переводим в эти символы, используя их, как  индексы массива, получаем «wMHCw8Q». Остается только добавить в конце один символ "=", как указание на один лишний нулевой байт, который мы добавляли на первом шаге и получить окончательный результат:

«АБВГД»: base64 = «wMHCw8Q=».

В общем случае для того, чтобы преобразовать данные, первый байт
помещается в самые старшие восемь бит 24-битного буфера, следующий — в
средние восемь и третий — в младшие значащие восемь бит. Если
кодируется менее чем три байта, то соответствующие биты буфера
устанавливаются в ноль. Далее каждые шесть бит буфера, начиная с самых
старших, используются как индексы строки

«ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz0123456789+/»

и её символы, на которые указывают индексы, помещаются в
выходную строку. Если кодируются только один или два байта, в результате
получаются только первые два или три символа строки, а выходная строка
дополняется двумя или одним символами «=». Это предотвращает
добавление дополнительных битов к восстановленным данным. Процесс
повторяется над оставшимися входными данными.
#### Декодирование (0.2)
С декодированием практически также легко. По сути это обратная операция
кодированию. Последовательность символов, полученных при конвертации
байт, мы разбиваем на ровные группы по 4. Затем каждый символ в
соответствии с алфавитом кодирования мы получаем цифровой порядковый
индекс (номер), каждое подобное значения мы конвертируем в двоичную
систему (6 бит) и получаем 24 бита, которые делим на уже три части и это
будут наши первоначальные байты информации. Повторить до конечного результата.
#### Протестируйте результат (0.1)
Протестируйте реализацию на 5-10 строках из ваших любимых текстов. Сравните результат с библиотекой base64.

In [1]:
import string
import base64

In [2]:
STRING = string.ascii_uppercase + string.ascii_lowercase + string.digits + "+/"

In [3]:
class EncodeDecodeWin1251:

    @staticmethod
    def encode(strings):
        bytes_list = list(map(lambda byte: format(byte, '08b'), strings.encode('windows-1251')))
        num_bytes_to_add = (3 - (len(strings) % 3)) % 3
        bits_string = "".join(bytes_list) + (num_bytes_to_add * 8 * '0')
        
        index = len(bits_string) - 6 * num_bytes_to_add
        encoded_string = "".join([STRING[int(bits_string[i:i+6], base=2)] for i in range(0, index, 6)]) \
        + num_bytes_to_add * '='
        
        return bytes(encoded_string, encoding='windows-1251')
    
    @staticmethod
    def decode(encoded_bytes):
        encoded_string = encoded_bytes.decode('windows-1251')
        initial_bytes_list = []
        
        for idx in range(0, len(encoded_string), 4):
            batch_bytes_list = [format(EncodeDecodeWin1251.get_index(symbol), '06b')
                                for symbol in encoded_string[idx:idx+4]]
            batch_bytes = "".join(batch_bytes_list)
        
            initial_bytes_list.extend([int(batch_bytes[i:i+8], base=2)
                                       for i in range(0, len(batch_bytes), 8)])
        
        initial_bytes_list = list(filter(lambda x: x>0, initial_bytes_list))
        return bytes(initial_bytes_list).decode('windows-1251')
        
    @staticmethod
    def get_index(symbol):
        if symbol == '=':
            return 0
        else: 
            return STRING.index(symbol)

In [4]:
texts = ['Сегодня лекции по слупам не будет',
        'Ребята, правильно ли я понимаю, что таблица со случайными процессами актуальная и мы можем закрыть этот вопрос.',
        'Осталось совсем немного времени, чтобы податься на Стартап-трек! Поторопитесь, в некоторых проектах очень нужны студенты.', 
        'Друзья, пожалуйста, перестаньте писать с просьбой заменить курс по выбору на кафедральный. На кафедре можно выбирать курсы.',
        'Экспресс-опрос на тему того, что именно вас сегодня заинтересовало']

In [5]:
for text in texts:
    print("My own encoding: {}".format(EncodeDecodeWin1251.encode(text)))
    print("base64 encoding: {}".format(base64.b64encode(text.encode('windows-1251'))))
    print("Result of decoding an incoded text: {}".format(
        EncodeDecodeWin1251.decode(EncodeDecodeWin1251.encode(text))))
    print("Check if the decoding of incoded text is correct: {}\n".format(
        EncodeDecodeWin1251.decode(EncodeDecodeWin1251.encode(text)) == text))

My own encoding: b'0eXj7uTt/yDr5er26Ogg7+4g8evz7+DsIO3lIOHz5OXy'
base64 encoding: b'0eXj7uTt/yDr5er26Ogg7+4g8evz7+DsIO3lIOHz5OXy'
Result of decoding an incoded text: Сегодня лекции по слупам не будет
Check if the decoding of incoded text is correct: True

My own encoding: b'0OXh//LgLCDv8ODi6Ov87e4g6+gg/yDv7u3o7OD+LCD38u4g8uDh6+j24CDx7iDx6/P34Ont++zoIO/w7vbl8fHg7Ogg4Ory8+Dr/O3g/yDoIOz7IOzu5uXsIOfg6vD78vwg/fLu8iDi7u/w7vEu'
base64 encoding: b'0OXh//LgLCDv8ODi6Ov87e4g6+gg/yDv7u3o7OD+LCD38u4g8uDh6+j24CDx7iDx6/P34Ont++zoIO/w7vbl8fHg7Ogg4Ory8+Dr/O3g/yDoIOz7IOzu5uXsIOfg6vD78vwg/fLu8iDi7u/w7vEu'
Result of decoding an incoded text: Ребята, правильно ли я понимаю, что таблица со случайными процессами актуальная и мы можем закрыть этот вопрос.
Check if the decoding of incoded text is correct: True

My own encoding: b'zvHy4Ovu8fwg8e7i8eXsIO3l7O3u4+4g4vDl7OXt6Cwg9/Lu4fsg7+7k4PL88f8g7eAg0fLg8PLg7y3y8OXqISDP7vLu8O7v6PLl8fwsIOIg7eXq7vLu8Pv1IO/w7uXq8uD1IO735e38IO3z5u37IPHy8+Tl7fL7Lg=='
base64 encoding: 